In [501]:
import random

def generate_leaf_scores(n):
    if n % 2 != 0:
        raise ValueError("Number of leaf nodes must be even for a balanced binary tree")

    leaf_scores = [random.randint(0, 100) for _ in range(n)]
    return leaf_scores

# Generate leaf scores with k symmetry
def generate_leaf_scores_with_symmetry(n, k):
    if n % 2 != 0:
        raise ValueError("Number of leaf nodes must be even for a balanced binary tree")

    if k < 0 or k > n // 2:
        raise ValueError("Invalid symmetry value")

    leaf_scores = []

    for _ in range(n - k * 2):
        score = random.randint(0, 100)
        leaf_scores.append(score)

    for _ in range(k):
        score = random.randint(0, 100)
        leaf_scores.append(score)
        leaf_scores.append(score)

    random.shuffle(leaf_scores)
    return leaf_scores

In [502]:
class TreeNode:
    def __init__(self, score):
        self.score = score
        self.active = True

    # These can actually be replaced with more complex logic representing the game in practice
    def compute_score(self):
        return self.score

    def is_symmetric(self):
        return self.active

DEFAULT_SCORE = -float("inf")

In [503]:
# Construct the tree (in array form) from the leaf scores, initialize the nodes with -1 as value
def construct_tree_from_leaf_scores(leaf_scores: list[int]) -> list[TreeNode]:
    n = len(leaf_scores)
    tree = [TreeNode(DEFAULT_SCORE) for _ in range(2 * n - 1)]
    for i in range(n):
        tree[n - 1 + i].score = leaf_scores[i]
    return tree

In [504]:
from treelib import Tree

def visualize_tree(tree_nodes: list[TreeNode]):
    # Visualize the tree in this pattern, use index as node id
    tree = Tree()
    n = len(tree_nodes)

    # Create nodes
    for i in range(n):
        parent_node_id = (i - 1) // 2
        tree_node = tree_nodes[i]
        label = f"{tree_node.score} ({'A' if tree_node.active else 'I'})"
        # print(parent_node_id)
        tree.create_node(label, i, parent=parent_node_id if parent_node_id >= 0 else None)

    print(tree.show(stdout=False))

In [505]:
def deactivate_symmetry(tree: list[TreeNode]):
    # Iterate from leaf nodes, if any nodes repeat, deactivate the node (the original node is not deactivated)
    # If both nodes are deactivated, then also deactivate the parent node
    # ! This will modify the tree in place

    value_lookup_map: dict[int, bool] = {}

    n = len(tree)
    for i in range(n - 1, -1, -1):
        # First check if the node score is already present in the map
        score = tree[i].score

        if score != DEFAULT_SCORE:
            if value_lookup_map.get(score, False):
                tree[i].active = False
            value_lookup_map[score] = True

        # If both children are deactivated, then also deactivate the parent node
        # left_child = i * 2 + 1
        # right_child = i * 2 + 2
        # if left_child < n and right_child < n:
        #     if not tree[left_child].active and not tree[right_child].active:
        #         tree[i].active = False


In [506]:
def negamax(tree: list[TreeNode], node_id: int, color: int) -> int:
    left_child = node_id * 2 + 1
    right_child = node_id * 2 + 2

    cur_node = tree[node_id]
    if left_child >= len(tree):
        # Leaf node
        return color * cur_node.compute_score()

    best_value = -float("inf")
    for i in [left_child, right_child]:
        value = -negamax(tree, i, -color)
        best_value = max(best_value, value)

    return int(best_value)

def negamax_with_symmetry(tree: list[TreeNode], node_id: int, color: int) -> int:
    left_child = node_id * 2 + 1
    right_child = node_id * 2 + 2

    cur_node = tree[node_id]
    if left_child >= len(tree):
        # Leaf node
        return color * cur_node.compute_score()

    best_value = -float("inf")
    for i in [left_child, right_child]:
        child_node = tree[i]
        if child_node.is_symmetric():
            value = -negamax_with_symmetry(tree, i, -color)
        else:
            value = color * child_node.compute_score()
        best_value = max(best_value, value)

    return int(best_value)

def negamax_with_alpha_beta_pruning(tree: list[TreeNode], node_id: int, color: int, alpha: int, beta: int) -> int:
    left_child = node_id * 2 + 1
    right_child = node_id * 2 + 2

    cur_node = tree[node_id]
    if left_child >= len(tree):
        # Leaf node
        return color * cur_node.compute_score()

    best_value = -float("inf")
    for i in [left_child, right_child]:
        value = -negamax_with_alpha_beta_pruning(tree, i, -color, -beta, -alpha)
        best_value = max(best_value, value)
        alpha = max(alpha, value)
        if alpha >= beta:
            break

    return int(best_value)

def negamax_with_alpha_beta_pruning_with_symmetry(tree: list[TreeNode], node_id: int, color: int, alpha: int, beta: int) -> int:
    left_child = node_id * 2 + 1
    right_child = node_id * 2 + 2

    cur_node = tree[node_id]
    if left_child >= len(tree):
        # Leaf node
        return color  * cur_node.compute_score()

    best_value = -float("inf")
    for i in [left_child, right_child]:
        child_node = tree[i]
        if child_node.is_symmetric():
            value = -negamax_with_alpha_beta_pruning_with_symmetry(tree, i, -color, -beta, -alpha)
            alpha = max(alpha, value)
            best_value = max(best_value, value)
            if alpha >= beta:
                break
        else:
            value = color * child_node.compute_score()
            best_value = max(best_value, value)

    return int(best_value)

In [507]:
import time

def run_and_measure(tree: list[TreeNode], runner: callable):
    start = time.time()
    result = runner()
    end = time.time()
    return result, end - start

def run_all(tree: list[TreeNode], log_output = False):
    if log_output:
        print("Running negamax...")
    result_negamax, time_negamax = run_and_measure(tree, lambda: negamax(tree, 0, 1))
    if log_output:
        print(f"Time: {time_negamax:.5f}s, Result: {result_negamax}")

    if log_output:
        print("Running negamax with symmetry...")
    result_negamax_symmetry, time_negamax_symmetry = run_and_measure(tree, lambda: negamax_with_symmetry(tree, 0, 1))
    if log_output:
        print(f"Time: {time_negamax_symmetry:.5f}s, Result: {result_negamax_symmetry}")

    if log_output:
        print("Running negamax with alpha-beta pruning...")
    result_negamax_alpha_beta, time_negamax_alpha_beta = run_and_measure(tree, lambda: negamax_with_alpha_beta_pruning(tree, 0, 1, -float("inf"), float("inf")))
    if log_output:
        print(f"Time: {time_negamax_alpha_beta:.5f}s, Result: {result_negamax_alpha_beta}")

    if log_output:
        print("Running negamax with alpha-beta pruning with symmetry...")
    result_negamax_alpha_beta_symmetry, time_negamax_alpha_beta_symmetry = run_and_measure(tree, lambda: negamax_with_alpha_beta_pruning_with_symmetry(tree, 0, 1, -float("inf"), float("inf")))
    if log_output:
        print(f"Time: {time_negamax_alpha_beta_symmetry:.5f}s, Result: {result_negamax_alpha_beta_symmetry}")

    return { "result_negamax": result_negamax,
            "result_negamax_symmetry": result_negamax_symmetry,
            "result_negamax_alpha_beta": result_negamax_alpha_beta,
            "result_negamax_alpha_beta_symmetry": result_negamax_alpha_beta_symmetry,
            "time_negamax": time_negamax,
            "time_negamax_symmetry": time_negamax_symmetry,
            "time_negamax_alpha_beta": time_negamax_alpha_beta,
            "time_negamax_alpha_beta_symmetry": time_negamax_alpha_beta_symmetry
        }


In [508]:
# leaf_scores = generate_leaf_scores_with_symmetry(102400, random.randint(0, 51200))
# leaf_scores = generate_leaf_scores(4)
# print(leaf_scores)
# Normal test cases
# leaf_scores = [-3,7,2,-1,-7,-3,8,4]
# leaf_scores = [9, -6, -4, -3]
# leaf_scores = [4,8,9,3,2,-2,9,-1]
# leaf_scores = [38,75, 87,9]

# Error test cases
# leaf_scores = [31, 34, 31, 93]  # Had diff result on symmetry
# leaf_scores = [6, 93, 6, 93] # Returned -inf for symmetry
# leaf_scores = [42, 71, 1, 12] # Crash because of inf on alpha-beta pruning with symmetry
leaf_scores = [27, 23, 17, 27] # Symmetry produces wrong result

full_tree = construct_tree_from_leaf_scores(leaf_scores)
deactivate_symmetry(full_tree)

visualize_tree(full_tree)

run_all(full_tree)

-inf (A)
├── -inf (A)
│   ├── 23 (A)
│   └── 27 (I)
└── -inf (A)
    ├── 17 (A)
    └── 27 (A)



{'result_negamax': 23,
 'result_negamax_symmetry': 23,
 'result_negamax_alpha_beta': 23,
 'result_negamax_alpha_beta_symmetry': 23,
 'time_negamax': 0.0,
 'time_negamax_symmetry': 0.0,
 'time_negamax_alpha_beta': 0.0,
 'time_negamax_alpha_beta_symmetry': 0.0}

### Testing

In [513]:
def run_test_comparer(tree: list[TreeNode]):
    try:
        run_result = run_all(tree)
        result_negamax = run_result["result_negamax"]
        result_negamax_symmetry = run_result["result_negamax_symmetry"]
        result_negamax_alpha_beta = run_result["result_negamax_alpha_beta"]
        result_negamax_alpha_beta_symmetry = run_result["result_negamax_alpha_beta_symmetry"]

        if result_negamax != result_negamax_symmetry or result_negamax != result_negamax_alpha_beta or result_negamax != result_negamax_alpha_beta_symmetry:
            print("Results are not equal", result_negamax, result_negamax_symmetry, result_negamax_alpha_beta, result_negamax_alpha_beta_symmetry)
            return False
        return True
    except Exception as e:
        return False

def run_test():
    NUM_TEST = 100
    print("Running tests...")

    pass_count = 0
    for _ in range(NUM_TEST):
        leaf_scores = generate_leaf_scores_with_symmetry(10000, random.randint(0, 5000))
        # leaf_scores = generate_leaf_scores_with_symmetry(4, random.randint(0, 2))
        full_tree = construct_tree_from_leaf_scores(leaf_scores)
        deactivate_symmetry(full_tree)

        if run_test_comparer(full_tree):
            pass_count += 1
        else:
            print("--- Failed on", leaf_scores)

    print(f"Passed {pass_count} tests out of {NUM_TEST}")

def run_known_suite():
    suites = [
        # Normal test cases
        [-3, 7, 2, -1, -7, -3, 8, 4],
        [9, -6, -4, -3],
        [4, 8, 9, 3, 2, -2, 9, -1],
        [38, 75,  87, 9],
        # Error test cases
        [31, 34, 31, 93],   # Had diff result on symmetry
        [6, 93, 6, 93],     # Returned -inf for symmetry
        [42, 71, 1, 12],    # Crash because of inf on alpha-beta pruning with symmetry
        [27, 23, 17, 27]    # Symmetry produces wrong result
    ]

    pass_count = 0
    for leaf_scores in suites:
        full_tree = construct_tree_from_leaf_scores(leaf_scores)
        deactivate_symmetry(full_tree)
        if run_test_comparer(full_tree):
            pass_count += 1
        else:
            print("--- Fail on", leaf_scores)

    print(f"Passed {pass_count} tests out of {len(suites)}")


In [511]:
run_test()

Running tests...
Passed 100 tests out of 100


In [512]:
run_known_suite()

Passed 8 tests out of 8
